<a href="https://colab.research.google.com/github/AARABAabdallah/RNAclassification/blob/master/RNA_Embedding_with_padding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
from tqdm import tnrange
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pickle

In [0]:
data = pd.read_csv('drive/My Drive/Colab Workspace/RNA_classification/DataSet/All_RNA_Families_IdSeqTotal_IdFamily_Seq.csv')
#Split data
X = np.array(data.sequence)
y = np.array(data.id_family)
family_labels = np.unique(np.array(y))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)
#Données de validations !
X_val1, X_val, y_val1, y_val = train_test_split(X_test, y_test, test_size=0.17)
#len(X_val)

In [0]:
X_train_copy = np.copy(X_train)
X_test_copy = np.copy(X_test)
X_val = np.copy(X_val)

In [0]:
X_train = np.copy(X_train_copy)
X_test = np.copy(X_test_copy)
X_val = np.copy(X_val)

In [26]:
X_train[0]

'AAUAAAAUCUCCUGAACAGGAGAGGGCCGACUGUUGCCUGAAAAAGGGAUCA'

In [27]:
# zero padding X_train pour booster l'entrainement
train_max_sequence = np.max([len(X_train[i]) for i in range(len(X_train))])
train_position_max_sequence = np.argmax([len(X_train[i]) for i in range(len(X_train))])
print("train_max_sequence: ",train_max_sequence)
print("train_position_max_sequence: ",train_position_max_sequence)
### Zero padding validation ###
val_max_sequence = np.max([len(X_val[i]) for i in range(len(X_val))])
val_position_max_sequence = np.argmax([len(X_val[i]) for i in range(len(X_val))])
print("val_max_sequence: ",val_max_sequence)
print("val_position_max_sequence: ",val_position_max_sequence)

train_max_sequence:  4031
train_position_max_sequence:  26344
val_max_sequence:  3937
val_position_max_sequence:  2560


In [0]:
### Pading data strings ###
def zero_padding_training_data_string(data, max_seq):
  #temp = np.copy(data)
  for i in tnrange(0, len(data)):
    data[i] = data[i].ljust(max_seq, '0')
  #return temp

In [29]:
### Pading training data strings X_train###
zero_padding_training_data_string(X_train, max_seq=train_max_sequence)

In [30]:
X_train[0]

'AAUAAAAUCUCCUGAACAGGAGAGGGCCGACUGUUGCCUGAAAAAGGGAUCA00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [31]:
### Pading validation data strings X_train###
zero_padding_training_data_string(X_val, val_max_sequence)

In [32]:
X_val.shape

(3063,)

In [0]:
def strings_to_numbers(data):
  output = []
  for i in tnrange(len(data)):
    string_to_number_array = []
    for l in list(data[i]):
      if l == 'A':
        string_to_number_array.append([1])
      elif l == 'C':
        string_to_number_array.append([2])
      elif l == 'G':
        string_to_number_array.append([3])
      elif l == 'T':
        string_to_number_array.append([4])
      else:
        string_to_number_array.append([0])
    
    output.append(np.array(string_to_number_array))
  return np.array(output)

In [34]:
X_train = strings_to_numbers(X_train)
X_test = strings_to_numbers(X_test)
X_val = strings_to_numbers(X_val)

In [47]:
X_train.shape

(54039, 4031, 1)

In [0]:
y_train_copy = np.copy(y_train)
y_test_copy = np.copy(y_test)
y_val_copy = np.copy(y_val)

In [0]:
y_train = np.copy(y_train_copy)
y_test = np.copy(y_test_copy)
y_val = np.copy(y_val_copy)

In [38]:
print(len(np.unique(y_train)))
print(len(np.unique(y_val)))

3016
1140


In [0]:
# OneHot Encoding labels: Just for now !!!
def one_hot_encode_labels(labels):
  output = []
  for i in tnrange(len(labels)):
    output_i = np.zeros(len(family_labels), dtype=int)
    output_i[labels[i]] = 1
    output.append(output_i)
  return np.array(output)

In [40]:
y_train = one_hot_encode_labels(y_train)
y_test = one_hot_encode_labels(y_test)
y_val = one_hot_encode_labels(y_val)

# Model Training

In [45]:
# Model Creation
from keras.models import Sequential
from keras.layers import LSTM, Input, Dense, Activation, Embedding, Bidirectional, Dropout

model = Sequential()
model.add(Embedding(input_dim=4, output_dim=64, input_length=None))
model.add(Bidirectional(LSTM(units=64, return_sequences=False, input_shape=(None, 64))))
model.add(Dropout(0.5))
model.add(Dense(len(family_labels),name='out_layer', activation='softmax'))
model.summary(90)

Model: "sequential_5"
__________________________________________________________________________________________
Layer (type)                            Output Shape                        Param #       
embedding_5 (Embedding)                 (None, None, 64)                    256           
__________________________________________________________________________________________
bidirectional_3 (Bidirectional)         (None, 128)                         66048         
__________________________________________________________________________________________
dropout_2 (Dropout)                     (None, 128)                         0             
__________________________________________________________________________________________
out_layer (Dense)                       (None, 3016)                        389064        
Total params: 455,368
Trainable params: 455,368
Non-trainable params: 0
_______________________________________________________________________________________

In [51]:
#model.compile(loss='categorical_crossentropy', optimizer='adam')
#history = model.fit(train_generator_embedding_layer(), steps_per_epoch=len(X_train_init), epochs=10, verbose=1)#, validation_data=(X_val, y_val))
X_train_init = X_train.reshape(len(X_train), train_max_sequence)
y_train_init = y_train
X_val_init = X_val.reshape(len(X_val), val_max_sequence)
y_val_init = y_val
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x=X_train_init, y=y_train_init, validation_data=(X_val_init, y_val_init) ,epochs=10, verbose=1)

Train on 54039 samples, validate on 3063 samples
Epoch 1/10
 2816/54039 [>.............................] - ETA: 1:56:36 - loss: 7.4288 - acc: 0.0227

KeyboardInterrupt: ignored

In [0]:
def compute_accuracy(data, labels):
  acc = 0
  for i in tnrange(len(data)):
    if np.argmax(model.predict(data[i].reshape(1, len(data[i]), 4))) == np.argmax(labels[i]):
      acc += 1
  return (acc/len(data))*100

def compute_loss(data, labels):
  loss = 0
  for i in tnrange(len(data)):
    x = data[i].reshape(1, len(data[i]), 4)
    y = labels[i].reshape(1,len(labels[i]))
    loss += model.evaluate(x, y, verbose=0)
  return loss/len(data)


In [0]:
def plot_history(history):
  # Plot training & validation accuracy values
  plt.plot(history['acc'])
  plt.plot(history['val_acc'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(history['loss'])
  plt.plot(history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.show()

In [0]:
history = train_epochs(n_epochs=6,already_trained=True)
plot_history(history.history)

Epoch 1/1
11268/54039 [=====>........................] - ETA: 45:47 - loss: 3.6552

In [0]:
history_file_name = 'drive/My Drive/Colab Workspace/RNA_classification/models/trainHistoryDict_Embedding'
previous_history = pickle.load(open(history_file_name, "rb"))
#plot_history(previous_history)
previous_history
#len(previous_history['val_loss'])
#print("number of epochs is:",len(history.history['loss']))
#history.history

FileNotFoundError: ignored

In [0]:
#Test accuracy
accuracy = compute_accuracy(X_test, y_test)
print("the model's accuracy is: ", accuracy, "%")


the model's accuracy is:  54.2275023594071 %
